In [38]:
from nltk.tokenize.stanford_segmenter import StanfordSegmenter
from nltk.parse import CoreNLPParser
from nltk.parse import corenlp
from nltk.tag import StanfordNERTagger
from nltk.tag import StanfordPOSTagger
from nltk.parse.stanford import StanfordParser
from nltk import Tree
from nltk.parse.stanford import StanfordDependencyParser
import jieba
from collections import Counter
import os

os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk-11.0.2.jdk/Contents/Home/bin/java" 
os.environ["CLASSPATH"] = "/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/jars"
os.environ["STANFORD_MODELS"] = "/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/models"

### Input reading
f = open("女水妖.txt", "r")

#==========抽取元素==========
## NN_Default
NN_except=['個','誰','時','兩','一','房子']
vocal = ['砰','咚']

loc = ['稻草屋','水潭','城堡','木屋子','磚屋','森林','高塔','河','奶奶家','王宮','花園','廚房','家','井里','山']

char_except = ['好孩子','身邊說','人們心中', '美好願望', '好幾', '個個', '小女兒', '美如天仙','小公主','嚴冬時節', '大雪片', '窗子邊', '針線活兒', '寒風捲著', '捲著雪片', '烏木窗臺', '針口流了', '三點血滴', '飄進窗子', '鮮紅血滴', '烏木窗臺', '血一樣', '這窗子','他們','你們','我們','弟弟','兄妹','女水妖','斧子']
family_member = ['祖父','祖母','爺爺','奶奶','外公','外婆','爸爸','媽媽','叔叔','舅舅','阿姨','嬸嬸','後母','姊姊','姐姐','繼母','姊妹','大哥','二哥','小弟']
passerby = ['公主','王子','女巫','狐狸','國王','皇后','獵人','王后','小矮人','魔鏡','野狼','妖精','精靈','僕人','灰姑娘','大臣','水妖','男孩']

Time = ['清晨','白天','黃昏','晚上']

## Character
### 1. 文章內所有角色
temp = []
char=[]
w = []
t = []


for x in f:
    
    seg_list = jieba.cut(x)
    tokens = "/".join(seg_list)
    tokenizer = tokens.split("/")
    #print(tokenizer)

    zh_tagger = StanfordPOSTagger('chinese-nodistsim.tagger')
    #print(zh_tagger.tag(tokens.split("/")))
    for _, word_and_tag in  zh_tagger.tag(tokens.split("/")):
        word, tag = word_and_tag.split('#')
        #print(word,tag)
        w.append(word)
        t.append(tag)
        
for i in range(1,len(w)):
    if(t[i-1]=="NN" and t[i]=="NN"):
        a = w[i-1]+w[i]
        if(a not in char):
            temp.append(a)
    
    elif(t[i-1]=="JJ" and t[i]=="NN"):
        a = w[i-1]+w[i]
        if(a not in char):
            temp.append(a)
    if(t[i]=="NN" or t[i]=="NR"):
        a = w[i]
        if(a not in char):
            temp.append(a)

i = 0
while i < len(temp):
    if temp[i] in char_except:
        del temp[i]
    else:
        i += 1
            
#print(temp)

### 2. 找出親人關係
for i in range(len(temp)):
    if(temp[i] in family_member and temp[i] not in char):
        char.append(temp[i])
#print(char)

### 3. 找出第三方角色
for i in range(len(temp)):
    if(temp[i] in passerby and temp[i] not in char):
        char.append(temp[i])
#print(char)

### 4. 計算重要角色 Top3
from collections import Counter
word_counts = Counter(temp)
top_three = word_counts.most_common(3)
#print(top_three)

### 5. 增加可能重要或未偵測出的角色
for i in range(len(top_three)):
    if(top_three[i][0] not in loc and top_three[i][0] not in family_member and top_three[i][0] not in char and top_three[i][0] not in NN_except):
        char.insert(0+i, top_three[i][0])
#print(char)

### 6. 移除重複
i = 0
while i < len(char):
    if char[i] in char_except:
        del char[i]
    else:
        i += 1

### character output
print("Character: ")
print(char)

## Location
### 1. 從名詞（temp）去查找對照的 loc_default，並存在 loc_temp
from collections import Counter

loc_temp = []

for i in range(len(temp)):
    if(temp[i] in loc):
        loc_temp.append(temp[i])

word_counts = Counter(loc_temp)
        
#print(loc_temp)

### 2. 從 loc_temp 整理
word_counts = Counter(loc_temp)
loc_counts = word_counts.most_common(len(loc_temp))
#print(loc_counts)

loc_set = []

for i in range(len(loc_counts)):
    loc_set.append(loc_counts[i][0])

### Location output
print("Location: ")
print(loc_set)

Character: 
['水妖', '男孩', '小姑娘']
Location: 
['井里', '山']


In [39]:
f = open("女水妖.txt","r")
outside = 0
inside = 0

for x in f:
    if("「" in x):
        for i in range(len(loc_set)):
            if(loc_set[i] in x):
                inside+=1
    else:
        for i in range(len(loc_set)):
            if(loc_set[i] in x):
                outside+=1

print((inside,outside))

if(inside>=outside): # "「" in x
    inside = True
else: # "「" not in x
    inside = False

(1, 5)


In [40]:
#==========生成劇本==========
# Latest Version

last_loc = ""
last_time=""
start=True
detect_print=True

### Input reading
f = open("女水妖.txt", "r")

for x in f:
    
    # Time
    # Night
    if('天黑' in x or '晚上' in x):
        time = Time[3]
    
    #evening    
    elif('黃昏' in x or '夕陽' in x):
        time = Time[2]
        
    #morning
    elif('晨' in x):
        time = Time[0]
        
    # default: 白天
    else:
        time = Time[1]
        
    #Location
    if(inside==True):
        if('「'  in x):
            for i in range(len(loc_set)): 
            # 地點 case1: 一開始
                if(loc_set[i] in x and loc_set[i]!=last_loc and start==True):
                    print("================================")
                    print("Scene: 時/ "+time+"     景/ "+loc_set[i])
                    last_loc = loc_set[i]
                    last_time = time
                    start=False
                    break
                
                if(start==True):
                    print("================================")
                    last_loc = '家'
                    print("Scene: 時/ "+time+"     景/ "+last_loc)
                    start = False
                    break
            
            # 地點 case2: 家以外的場景
                if(loc_set[i] in x and loc_set[i]!=last_loc and loc_set[i]!="家"):
                    print("================================")
                    print("Scene: 時/ "+time+"     景/ "+loc_set[i])
                    last_loc = loc_set[i]
                    last_time = time
                    break
                
                '''if(time!=last_time):
                    print("================================")
                    print("Scene: 時/ "+time+"     景/ "+last_loc)
                    last_time = time'''
    
    else: 
        #outside
        if('「'  not in x):
            for i in range(len(loc_set)): 
            # 地點 case1: 一開始
                if(loc_set[i] in x and loc_set[i]!=last_loc and start==True):
                    print("================================")
                    print("Scene: 時/ "+time+"     景/ "+loc_set[i])
                    last_loc = loc_set[i]
                    last_time = time
                    start=False
                    break
                
                if(start==True):
                    print("================================")
                    last_loc = '家'
                    print("Scene: 時/ "+time+"     景/ "+last_loc)
                    start = False
                    break
            
            # 地點 case2: 家以外的場景
                if(loc_set[i] in x and loc_set[i]!=last_loc and loc_set[i]!="家"):
                    print("================================")
                    print("Scene: 時/ "+time+"     景/ "+loc_set[i])
                    last_loc = loc_set[i]
                    last_time = time
                    break
                
                '''if(time!=last_time):
                    print("================================")
                    print("Scene: 時/ "+time+"     景/ "+last_loc)
                    last_time = time'''
            
    
        # 強迫換景
        if('隔天' in x or '第二天' in x):
            print("================================")
            print("Scene: 時/ "+time+"     景/ "+last_loc)
            last_time = time

       
    if("「" in x):
        # case 3-2
        if(x[0]=="「"):
            upper = (x.split("「"))
            two = upper[1]

            if(len(upper)>2):
                dia1 = upper[1].split("」")[0]
                sentence1 =  two.split("」")[1]
                sentence2 = sentence1
                dia2 = upper[2].split("」")[0]
                
                sentence = []
                sentence.append(sentence1)
                sentence.append(sentence2)

                dialogue = []
                dialogue.append(dia1)
                dialogue.append(dia2)

                for i in range(2):

                    seg_list = jieba.cut(sentence[i]) 
                    tokens = "/".join(seg_list)
                    tokenizer = tokens.split("/")
                    #print(tokenizer)
                    for j in range(len(char)):
                        if(char[j] in tokenizer):
                            A = char[j]
                            break
                    B=A

                    zh_dependency_parser = StanfordDependencyParser("/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/jars/stanford-parser.jar","/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/jars/stanford-parser-3.9.2-models.jar","/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/models/chineseFactored.ser.gz")
                    ans = list(zh_dependency_parser.parse(tokens.split("/")))
                    for row in ans[0].triples():
                        if(row[1]=="nsubj" and row[2][0] in char):
                            A = row[2][0]
                            #print("from: "+A)
                        elif(row[1]=="compound:nn" and row[2][0] in char_except):
                            B = row[2][0]
                            #print("to: "+B)    
                for k in range(len(vocal)):
                    if(vocal[i] in dialogue[i]):
                        detect_print = False
                        break
                
                if(detect_print):
                    print(A+": "+dialogue[i])
                    print(sentence[i])
                else:
                    detect_print = True
                
                
            # case 3-1
            else:
                dialogue = upper[1].split("」")[0]
                sentence = upper[1].split("」")[1]
                
                seg_list = jieba.cut(sentence) 
                tokens = "/".join(seg_list)
                tokenizer = tokens.split("/")
                #print(tokenizer)
                for j in range(len(char)):
                        if(char[j] in tokenizer):
                            A = char[j]
                            break
                B=A
                
                zh_dependency_parser = StanfordDependencyParser("/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/jars/stanford-parser.jar","/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/jars/stanford-parser-3.9.2-models.jar","/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/models/chineseFactored.ser.gz")
                ans = list(zh_dependency_parser.parse(tokens.split("/")))
                
                for row in ans[0].triples():
                    if(row[1]=="nsubj" and row[2][0] in char):
                        A = row[2][0]
                        #print("from: "+A)
                    elif(row[1]=="compound:nn" and row[2][0] in char_except):
                        B = row[2][0]
                        #print("to: "+B)

                for k in range(len(vocal)):
                    if(vocal[k] in dialogue):
                        detect_print = False
                        break
                
                if(detect_print):
                        print(A+": "+dialogue)
                        #print(sentence)
                else:
                    detect_print = True
                   
        # case 1, 2
        else:
            upper = (x.split("「"))
            two = upper[1]

            # case 2
            if(len(upper)>2):
                sentence1 = upper[0]
                dia1 =  upper[1].split("」")[0]
                sentence2 = (two.split("」"))[1]
                dia2 = upper[2].split("」")[0]
                
                sentence = []
                sentence.append(sentence1)
                sentence.append(sentence2)

                dialogue = []
                dialogue.append(dia1)
                dialogue.append(dia2)

                for i in range(2):

                    seg_list = jieba.cut(sentence[i]) 
                    tokens = "/".join(seg_list)
                    tokenizer = tokens.split("/")
                    #print(tokenizer)
                    for j in range(len(char)):
                        if(char[j] in tokenizer):
                            A = char[j]
                            break
                    B=A

                    zh_dependency_parser = StanfordDependencyParser("/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/jars/stanford-parser.jar","/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/jars/stanford-parser-3.9.2-models.jar","/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/models/chineseFactored.ser.gz")
                    ans = list(zh_dependency_parser.parse(tokens.split("/")))
                    for row in ans[0].triples():
                        if(row[1]=="nsubj" and row[2][0] in char):
                            A = row[2][0]
                            #print("from: "+A)
                        elif(row[1]=="compound:nn" and row[2][0] in char_except):
                            B = row[2][0]
                            #print("to: "+B)
                
                for k in range(len(vocal)):
                    if(vocal[k] in dialogue[i]):
                        detect_print = False
                        break
                
                if(detect_print):
                        print(sentence[i])
                        print(A+": "+dialogue[i])
                else:
                    detect_print = True
                
            # case 1
            else:
                sentence = upper[0]
                dialogue = upper[1].split("」")[0]
                
                seg_list = jieba.cut(sentence) 
                tokens = "/".join(seg_list)
                tokenizer = tokens.split("/")
                #print(tokenizer)
                for j in range(len(char)):
                        if(char[j] in tokenizer):
                            A = char[j]
                            break
                B=A

                zh_dependency_parser = StanfordDependencyParser("/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/jars/stanford-parser.jar","/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/jars/stanford-parser-3.9.2-models.jar","/Users/sumeiru/Desktop/StanfordNLP/StanfordNLP/models/chineseFactored.ser.gz")
                ans = list(zh_dependency_parser.parse(tokens.split("/")))
                
                for row in ans[0].triples():
                    if(row[1]=="nsubj" and row[2][0] in char):
                        A = row[2][0]
                        #print("from: "+A)
                    elif(row[1]=="compound:nn" and row[2][0] in char_except):
                        B = row[2][0]
                        #print("to: "+B)

                for k in range(len(vocal)):
                    if(vocal[k] in dialogue):
                        detect_print = False
                        break
                
                if(detect_print):
                        print(A+": "+dialogue)
                else:
                    detect_print = True
                
    else:
        print(x)
        
print("============The End============")

Scene: 時/ 白天     景/ 井里
從前有兄妹兩人在井邊玩耍，不小心掉進了井里。



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:263: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.StanforCoreNLPDependencyParser instead.


水妖: 現在你們可是在我的手裡了，替我好好幹活吧！
她給小姑娘一把亂糟糟的臟亞麻要她紡，給她一個漏了的水桶要她打水

男孩子則被迫去砍伐木頭，可斧子是鈍的，根本砍不動樹。

至於吃的，除了像石頭一般硬的面疙瘩就再也沒有別的了。

孩子們忍無可忍，在一個星期天，趁水妖上教堂的工夫，兄妹倆悄悄地溜走了。

水妖從教堂回來，發現小鳥們飛走了，撒腿就追。

Scene: 時/ 白天     景/ 山
遠遠看到水妖追來，姑娘便朝身後扔了一把刷子，刷子頓時變成了一座長滿荊棘的大山，水妖只好艱難地往上爬，終於爬過來了。

孩子們一看不好，男孩又扔出一把梳子，那梳子頓時變成成千上萬顆牙齒，可是水妖還是穩穩噹噹地跨過來了。

小姑娘又扔了一塊鏡子，鏡子變成一座光滑的山峰，任水妖怎麼爬也難以爬上來。

水妖: 還是快些回家象把斧子來把這玻璃山砍成兩半吧。
可是等她取來斧子把玻璃山砍開時，小兄妹早已逃得遠遠的了。

Scene: 時/ 白天     景/ 井里
女水妖只好又回到井里去了。
============The End============


## Debug

In [14]:
loc_set = ['稻草屋', '家', '木屋子', '磚屋']

In [17]:
f = open("青蛙王子.txt","r")
outside = 0
inside = 0

for x in f:
    print(x)
    if("「" in x):
        for i in range(len(loc_set)):
            if(loc_set[i] in x):
                inside+=1
    else:
        for i in range(len(loc_set)):
            if(loc_set[i] in x):
                outside+=1

print((inside,outside))

在遙遠的古代，人們心中的美好願望往往能夠變成現實。

就在那個令人神往的時代，曾經有過一位國王。

國王有好幾個女兒，個個都長得非常美麗；尤其是他的小女兒，更是美如天仙，就連見多識廣的太陽，每次照在她臉上時，都對她的美麗感到驚詫不已。

國王的宮殿附近，有一片幽暗的大森林。

在這片森林中的一棵老椴樹下，有一個水潭，水潭很深。

在天熱的時候，小公主常常來到這片森林，坐在清涼的水潭邊上。

她坐在那裡感到無聊的時候，就取出一隻金球，把金球拋向空中，然後再用手接住。

這成了她最喜愛的游戲。

不巧的是，有一次，小公主伸出兩隻小手去接金球，金球卻沒有落進她的手裡，而是掉到了地上，而且一下子就滾到了水潭裡。

小公主兩眼緊緊地盯著金球，可是金球忽地一下子在水潭裡就沒影兒了。

因為水潭裡的水很深，看不見底，小公主就哭了起來，她的哭聲越來越大，哭得傷心極了。

哭著哭著，小公主突然聽見有人大聲說：「哎呀，公主，您這是怎麼啦？您這樣嚎啕大哭，就連石頭聽了都會心疼的呀。」

聽了這話，小公主四處張望，想弄清楚說話聲是從哪兒傳來的，不料卻發現一隻青蛙，從水裡伸出他那醜陋不堪的肥嘟嘟的大腦袋。

「啊！原來是你呀，游泳健將」小公主對青蛙說道，「我在這兒哭，是因為我的金球掉進水潭裡去了。」

「好啦，不要難過，別哭了，」青蛙回答說，「我有辦法幫助您。要是我幫您把您的金球撈出來，您拿什麼東西來回報我呢？」

「親愛的青蛙，你要什麼東西都成呵，」小公主回答說，「我的衣服、我的珍珠和寶石、甚至我頭上戴著的這頂金冠，都可以給你。」

聽了這話，青蛙對小公主說：「您的衣服、您的珍珠、您的寶石，還有您的金冠，我哪樣都不想要。不過，要是您喜歡我，讓我做您的好朋友，我們一起游戲，吃飯的時候讓我和您同坐一張餐桌，用您的小金碟子吃東西，用您的小高腳杯飲酒，晚上還讓我睡在您的小床上；要是您答應所有這一切的話，我就潛到水潭裡去，把您的金球撈出來。」

「好的，太好了，」小公主說，「只要你願意把我的金球撈出來，你的一切要求我都答應。」

小公主雖然嘴上這麼說，心裡卻想：「這隻青蛙可真夠傻的，盡胡說八道！他只配蹲在水潭裡，和其他青蛙一起呱呱叫，怎麼可能做人的好朋友呢？」

青蛙得到了小公主的許諾之後，把腦袋往水裡一扎，就潛入了水潭。

過了不大一會兒，青蛙嘴裡銜著金球，浮出了水面，然後把金球吐在草地上。

小